In [1]:
from renishawWiRE import WDFReader

In [36]:
file = r"\\vm-stor-51\alle\Projekte Kunststoffe\RD\MA\Öffentliche_Projekte\2020-01-07_230007_H2020-CHARISMA_barton\Python\charisma-raman-spectrum-harmonization\test_data\200218-17.wdf"

In [43]:
filename, file_extension = os.path.splitext(file)

In [46]:
filename

'\\\\vm-stor-51\\alle\\Projekte Kunststoffe\\RD\\MA\\Öffentliche_Projekte\\2020-01-07_230007_H2020-CHARISMA_barton\\Python\\charisma-raman-spectrum-harmonization\\test_data\\200218-17'

In [45]:
file_extension

'.wdf'

In [42]:
file[:file.rfind(".")] + ".cha"

'\\\\vm-stor-51\\alle\\Projekte Kunststoffe\\RD\\MA\\Öffentliche_Projekte\\2020-01-07_230007_H2020-CHARISMA_barton\\Python\\charisma-raman-spectrum-harmonization\\test_data\\200218-17.cha'

In [3]:
s = WDFReader(file)

In [4]:
import numpy as np

In [6]:
y_data = s.spectra
x_data = s.xdata
#        counts = s.amplitudes
#        wavenumbers = s.wavelength
if np.mean(np.diff(x_data)) < 0:
    y_data = np.flip(y_data)
    x_data = np.flip(x_data)

In [7]:
import time

In [28]:
static_metadata = {
    "laser wavelength": s.laser_length,
    "no. of accumulations": s.accumulation_count,
    "spectral unit": s.spectral_unit.name,
    "OEM software name": s.application_name,
    "OEM software version": s.application_version
}

In [29]:
static_metadata["Generated on"] = time.ctime()

In [9]:
def getYDataType(y_data):
    types = {0: "Single spectrum", 1: "Line scan", 2: "Map", 3: "Map series / volume"}
    return types[len(y_data.shape)-1]

In [10]:
dynamic_metadata = {
"Raman data type": getYDataType(y_data),
"xy dimensions":  y_data.shape[1:],
"no. of channels": y_data.shape[0],
"minimum wavelength": x_data.min(),
"maximum wavelength": x_data.max(),
"mean counts": y_data.mean(),
"standard deviation": y_data.std(),  
}

In [11]:
import pandas as pd

In [12]:
pd.Series(dynamic_metadata)

Raman data type       Single spectrum
xy dimensions                      ()
no. of channels                  3190
minimum wavelength             102.77
maximum wavelength            3703.71
mean counts                   62918.4
standard deviation            27122.4
dtype: object

In [13]:
pd.Series(static_metadata)

laser wavelength                         514.501
no. of accumulations                           3
spectral unit                             Counts
OEM software name                           WiRE
OEM software version             [5, 2, 0, 9499]
Generated on            Fri Mar 19 19:39:08 2021
dtype: object

Transformer list is [ [Name1, p1_1, p1_2, ...], [Name2, p2_1, p2_2, ...] ]

In [14]:
transformers = [ ['t1', 24, 4, 1], ['t2', 2] ]

In [15]:
commits = ["Generated CHADA on " + time.ctime()]

Construct CHADA archive

In [23]:
import zipfile
import os

In [105]:
chada_path = r"C:\Users\barton\Desktop\chada"

In [106]:
zf = zipfile.ZipFile(chada_path + "//test.cha", mode="w", compression=zipfile.ZIP_DEFLATED)

Add native file

In [107]:
zf.write(file, os.path.basename(file))

Add data

In [108]:
zf.writestr("static_meta.txt", str(static_metadata))
zf.writestr("dynamic_meta.txt", str(dynamic_metadata))
zf.writestr("transformers.txt", str(transformers))
zf.writestr("commits.txt", str(commits))
zf.close()

In [111]:
zf = zipfile.ZipFile(chada_path + "//test.cha", "a")

In [112]:
zf.writestr("dynamic_meta.txt", "Hallo")
zf.close()

Read from CHADA archive

In [96]:
zf = zipfile.ZipFile(chada_path + "//chada.cha")

In [97]:
zf.close()

In [102]:
zf.open(chada_path + "//chada.cha")

ValueError: Attempt to use ZIP archive that was already closed

In [99]:
import ast

In [100]:
sm = ast.literal_eval(zf.read("static_meta.txt").decode('utf-8'))

ValueError: Attempt to use ZIP archive that was already closed

In [153]:
pd.Series(sm)

laser wavelength                         514.501
no. of accumulations                           3
spectral unit                             Counts
OEM software name                           WiRE
OEM software version             [5, 2, 0, 9499]
Generated on            Fri Mar 19 18:51:06 2021
dtype: object

In [154]:
t = ast.literal_eval(zf.read("transformers.txt").decode('utf-8'))

In [155]:
pd.Series(t)

0    [t1, 24, 4, 1]
1           [t2, 2]
dtype: object

In [75]:
extract_path = zf.extract(os.path.basename(file), os.path.dirname(file))

In [69]:
s1 = WDFReader(extract_path)

In [71]:
s1.xdata

array([3703.711   , 3702.8545  , 3701.999   , ...,  105.708984,
        104.240234,  102.76953 ], dtype=float32)

In [74]:
os.remove(r"\\vm-stor-51\alle\Projekte Kunststoffe\RD\MA\Öffentliche_Projekte\2020-01-07_230007_H2020-CHARISMA_barton\Python\charisma-raman-spectrum-harmonization\test_data\chada\200218-17.wdf")

In [77]:
locals()["getYDataType"]

<function __main__.getYDataType(y_data)>

In [91]:
class Foo():
    def __init__(self):
        return
    
    def bar(self):
        self.m = "Hallo"
        return "Hallo"
    
    def exec(self, which='bar'):
        getattr(self, which)()
        return

In [92]:
F = Foo()

In [93]:
F.m

AttributeError: 'Foo' object has no attribute 'm'

In [94]:
F.exec()

In [95]:
F.m

'Hallo'

In [113]:
y_data

array([28312.81 , 28992.941, 29635.139, ..., 91030.016, 90612.66 ,
       90308.695], dtype=float32)

In [114]:
np.savetxt(r"C:\Users\barton\Desktop\chada\baseline.txt", y_data)

In [115]:
np.loadtxt(r"C:\Users\barton\Desktop\chada\baseline.txt")

array([28312.81054688, 28992.94140625, 29635.13867188, ...,
       91030.015625  , 90612.65625   , 90308.6953125 ])

In [127]:
y_data

array([28312.81 , 28992.941, 29635.139, ..., 91030.016, 90612.66 ,
       90308.695], dtype=float32)

In [135]:
zf = zipfile.ZipFile(r"C:\Users\barton\Desktop\chada\test", mode="w", compression=zipfile.ZIP_DEFLATED)

In [136]:
np.savetxt(r"C:\Users\barton\Desktop\chada\baseline.txt", y_data)

In [137]:
zf.write(r"C:\Users\barton\Desktop\chada\baseline.txt", os.path.basename(r"C:\Users\barton\Desktop\chada\baseline.txt"))

In [138]:
zf.close()

In [139]:
zf = zipfile.ZipFile(r"C:\Users\barton\Desktop\chada\test")

In [145]:
zf.extract("baseline.txt", r"C:\Users\barton\Desktop\chada")

'C:\\Users\\barton\\Desktop\\chada\\baseline.txt'

In [146]:
b = np.loadtxt('C:\\Users\\barton\\Desktop\\chada\\baseline.txt')

In [147]:
b

array([28312.81054688, 28992.94140625, 29635.13867188, ...,
       91030.015625  , 90612.65625   , 90308.6953125 ])

In [148]:
zf.close()

In [149]:
os.remove('C:\\Users\\barton\\Desktop\\chada\\baseline.txt')

In [152]:
list = ["t", b.tolist()]

In [153]:
list

['t',
 [28312.810546875,
  28992.94140625,
  29635.138671875,
  30298.396484375,
  30645.921875,
  31111.650390625,
  32246.1953125,
  33540.9609375,
  34850.09375,
  36088.45703125,
  37033.54296875,
  37586.11328125,
  38149.3125,
  38510.99609375,
  38314.5,
  38622.4140625,
  38605.07421875,
  38408.484375,
  37945.51953125,
  37174.25390625,
  35967.41015625,
  34601.65234375,
  33379.96484375,
  32117.765625,
  30348.0546875,
  29036.044921875,
  27615.486328125,
  26365.263671875,
  25163.408203125,
  24658.724609375,
  23881.73828125,
  23027.705078125,
  22258.865234375,
  21599.697265625,
  21149.76171875,
  20683.947265625,
  20439.736328125,
  20033.0078125,
  19865.44921875,
  19561.55078125,
  18955.220703125,
  19330.947265625,
  18553.037109375,
  18495.20703125,
  18589.5,
  18701.31640625,
  18480.78515625,
  18356.408203125,
  18160.234375,
  18354.318359375,
  18334.888671875,
  18019.533203125,
  18033.30859375,
  17906.958984375,
  17673.693359375,
  17393.03125,


In [154]:
zf = zipfile.ZipFile(r"C:\Users\barton\Desktop\chada\test", mode="w", compression=zipfile.ZIP_DEFLATED)

In [155]:
zf.writestr("transformers.txt", str(list))

In [157]:
zf.close()

In [159]:
b1 = b.tolist()

In [160]:
b2 = np.array(b1)

In [161]:
b2

array([28312.81054688, 28992.94140625, 29635.13867188, ...,
       91030.015625  , 90612.65625   , 90308.6953125 ])

In [163]:
list.copy()

['t',
 [28312.810546875,
  28992.94140625,
  29635.138671875,
  30298.396484375,
  30645.921875,
  31111.650390625,
  32246.1953125,
  33540.9609375,
  34850.09375,
  36088.45703125,
  37033.54296875,
  37586.11328125,
  38149.3125,
  38510.99609375,
  38314.5,
  38622.4140625,
  38605.07421875,
  38408.484375,
  37945.51953125,
  37174.25390625,
  35967.41015625,
  34601.65234375,
  33379.96484375,
  32117.765625,
  30348.0546875,
  29036.044921875,
  27615.486328125,
  26365.263671875,
  25163.408203125,
  24658.724609375,
  23881.73828125,
  23027.705078125,
  22258.865234375,
  21599.697265625,
  21149.76171875,
  20683.947265625,
  20439.736328125,
  20033.0078125,
  19865.44921875,
  19561.55078125,
  18955.220703125,
  19330.947265625,
  18553.037109375,
  18495.20703125,
  18589.5,
  18701.31640625,
  18480.78515625,
  18356.408203125,
  18160.234375,
  18354.318359375,
  18334.888671875,
  18019.533203125,
  18033.30859375,
  17906.958984375,
  17673.693359375,
  17393.03125,


In [164]:
from specio import show_formats, specread

In [166]:
show_formats()

CSV - Internal Comma-Separated Values format [.csv]
DUMMY - An example format that does nothing. [.foobar, .nonexistentext]
FSM - FSM Perkin Elmer Spotlight IR instrument binary format [.fsm]
MZML - MZML Mass Spectrometry data format [.mzml]
MZXML - mzXML Mass Spectrometry data format [.mzxml]
SP - SP Perkin Elmer binary format. [.sp]
SPC - Galactic Industries Corporation binary format [.spc]


In [172]:
s = specread(r"\\vm-stor-51\alle\Projekte Kunststoffe\RD\MA\Öffentliche_Projekte\2020-01-07_230007_H2020-CHARISMA_barton\Python\charisma-raman-spectrum-harmonization\test_data\SILICON_x20_1%_02.spc")

gx-y(1)


In [173]:
s.amplitudes

array([118.69281769, 123.46276093, 113.72263336, ...,  36.72757339,
        37.0501976 ,  27.31775093])

In [174]:
s.wavelength

array([3501.18237305, 3499.8627279 , 3498.54308276, ...,  103.0961308 ,
        101.77648566,  100.45684052])

In [198]:
s.meta

{'length': 11751,
 'ftflg': b'\x00',
 'fversn': b'K',
 'fexper': 0,
 'fexp': 128,
 'fnpts': 2578,
 'ffirst': 3501.182373046875,
 'flast': 100.45684051513672,
 'fnsub': 1,
 'fxtype': 13,
 'fytype': 4,
 'fztype': 0,
 'fpost': b'\x00',
 'fdate': 2118914663,
 'fres': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 'fsource': b'WiRE2\x00\x00\x00\x00',
 'fpeakpt': 0,
 'fspare': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 'fcmnt': "b'A single scan measurement generated by the WiRE2 spectral acquisition wizard.\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'",
 'fcatxt': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 'f

In [200]:
a=b=c=0

In [201]:
a

0

In [202]:
b+=2

In [203]:
a

0

In [204]:
b

2

In [205]:
c

0

In [206]:
np.inf>90

True